In [1]:
from leaderboard import *

## Alle Parts 2 (Sekunden)

In [2]:
data = load_data(leaderboard_id=976545, year=2024)
data

,1,2,3,4,5,6,7
3309362,238,6094,<NA>,<NA>,<NA>,<NA>,<NA>
999754,321,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
attoPascal,303,2703,2189,1698,1558,36577,348
Belim,246,1864,839,928,1865,11227,742
Bene-dykt,1061,44036,382,1193,1777,4011,36294
berni664,494,4209,1856,1020,2395,8410,1107
cluosh,<NA>,<NA>,411,<NA>,<NA>,<NA>,<NA>
DavVader123,526,4812,903,2138,1949,4523,3618
emxchu,459,3418,29966,<NA>,<NA>,<NA>,<NA>
FelixSH,507,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


## Part 2 Leaderboard

In [3]:
points(data).head(20)

,pts,name
1,264,Vonuras
2,264,JustAlittleWolf
3,247,Salominius
4,226,memoriesadrift
5,223,msonnberger
6,219,Triarier
7,216,Belim
8,215,reputation
9,211,fwoerister
10,199,Raphael Tarita


## Sekunden für aktuellen Tag, Part 2

In [4]:
seconds_latest_day(data)

,7
JustAlittleWolf,79
Vonuras,94
Salominius,133
fwoerister,135
Triarier,197
reputation,316
attoPascal,348
Godototot,372
Belim,742
berni664,1107


## Minuten für alle Parts 2

In [5]:
sum_seconds(data) // 60

,0
Vonuras,50
JustAlittleWolf,66
memoriesadrift,194
Triarier,260
Belim,295
DavVader123,307
berni664,324
Salominius,349
michalrzak,463
fwoerister,600
